In [173]:
from xml.dom import minidom
import numpy as np
import heapq
import random
import matplotlib.pyplot as plt

In [221]:
data = minidom.parse('./bayg29.xml')
#parameters
initialPopulationCount=3000
numberOfCities=29
crossoverPossibility=0.95
mutationPossibility=0.10
maxFrozenIteration=50
elitismSetSize=40
tournamentSize=5


In [106]:
costs=[]
vertices=data.getElementsByTagName('vertex')
for vertex in vertices:
    cost=[]
    for edge in vertex.getElementsByTagName('edge'):
        if len(cost)==len(costs):
            cost.append(0);
        cost.append(float(edge.getAttribute('cost')))
    costs.append(cost)

        

In [107]:
#visualize distances in a DataFrame
from pandas import DataFrame
df=DataFrame(costs)
df

0      1      2      3      4      5      6      7      8      9   \
0     0.0   97.0  205.0  139.0   86.0   60.0  220.0   65.0  111.0  115.0   
1    97.0    0.0  129.0  103.0   71.0  105.0  258.0  154.0  112.0   65.0   
2   205.0  129.0    0.0  219.0  125.0  175.0  386.0  269.0  134.0  184.0   
3   139.0  103.0  219.0    0.0  167.0  182.0  180.0  162.0  208.0   39.0   
4    86.0   71.0  125.0  167.0    0.0   51.0  296.0  150.0   42.0  131.0   
5    60.0  105.0  175.0  182.0   51.0    0.0  279.0  114.0   56.0  150.0   
6   220.0  258.0  386.0  180.0  296.0  279.0    0.0  178.0  328.0  206.0   
7    65.0  154.0  269.0  162.0  150.0  114.0  178.0    0.0  169.0  151.0   
8   111.0  112.0  134.0  208.0   42.0   56.0  328.0  169.0    0.0  172.0   
9   115.0   65.0  184.0   39.0  131.0  150.0  206.0  151.0  172.0    0.0   
10  227.0  204.0  313.0  102.0  268.0  278.0  147.0  227.0  309.0  140.0   
11   95.0  150.0  201.0  227.0   88.0   46.0  308.0  133.0   68.0  195.0   
12   82.0   87.0  215.0   60.0  131.0  133.0  172.0  104.0  169.0   51.0   
13  225.0  176.0  267.0   86.0  245.0  266.0  203.0  242.0  286.0  117.0   
14  168.0  137.0  248.0   34.0  201.0  214.0  165.0  182.0  242.0   72.0   
15  103.0  142.0  271.0   96.0  175.0  162.0  121.0   84.0  208.0  104.0   
16  266.0  204.0  274.0  129.0  275.0  302.0  251.0  290.0  315.0  153.0   
17  205.0  148.0  236.0   69.0  218.0  242.0  216.0  230.0  259.0   93.0   
18  149.0  148.0  272.0   58.0  202.0  203.0  122.0  146.0  240.0   88.0   
19  120.0   49.0  160.0   60.0  119.0  146.0  231.0  165.0  160.0   25.0   
20   58.0   41.0  151.0  120.0   50.0   67.0  249.0  121.0   90.0   85.0   
21  257.0  211.0  300.0  119.0  281.0  300.0  209.0  270.0  322.0  152.0   
22  152.0  226.0  350.0  192.0  238.0  205.0  111.0   91.0  260.0  200.0   
23   52.0  116.0  239.0  114.0  131.0  111.0  169.0   48.0  160.0  104.0   
24  180.0  197.0  322.0  110.0  244.0  238.0   72.0  158.0  281.0  139.0   
25  136.0   89.0   78.0  192.0   51.0   98.0  338.0  200.0   57.0  154.0   
26   82.0  153.0  276.0  136.0  166.0  139.0  144.0   39.0  192.0  134.0   
27   34.0  124.0  220.0  173.0   95.0   52.0  237.0   64.0  107.0  149.0   
28  145.0   74.0   60.0  173.0   69.0  120.0  331.0  210.0   90.0  135.0   

    ...       19     20     21     22     23     24     25     26     27  \
0   ...    120.0   58.0  257.0  152.0   52.0  180.0  136.0   82.0   34.0   
1   ...     49.0   41.0  211.0  226.0  116.0  197.0   89.0  153.0  124.0   
2   ...    160.0  151.0  300.0  350.0  239.0  322.0   78.0  276.0  220.0   
3   ...     60.0  120.0  119.0  192.0  114.0  110.0  192.0  136.0  173.0   
4   ...    119.0   50.0  281.0  238.0  131.0  244.0   51.0  166.0   95.0   
5   ...    146.0   67.0  300.0  205.0  111.0  238.0   98.0  139.0   52.0   
6   ...    231.0  249.0  209.0  111.0  169.0   72.0  338.0  144.0  237.0   
7   ...    165.0  121.0  270.0   91.0   48.0  158.0  200.0   39.0   64.0   
8   ...    160.0   90.0  322.0  260.0  160.0  281.0   57.0  192.0  107.0   
9   ...     25.0   85.0  152.0  200.0  104.0  139.0  154.0  134.0  149.0   
10  ...    159.0  219.0   63.0  216.0  187.0   88.0  293.0  191.0  258.0   
11  ...    192.0  113.0  345.0  222.0  144.0  274.0  124.0  165.0   71.0   
12  ...     71.0   82.0  176.0  150.0   56.0  114.0  168.0   83.0  115.0   
13  ...    127.0  201.0   36.0  254.0  196.0  136.0  260.0  212.0  258.0   
14  ...     91.0  153.0   91.0  197.0  136.0   94.0  225.0  151.0  201.0   
15  ...    126.0  128.0  190.0   98.0   53.0   78.0  218.0   48.0  127.0   
16  ...    157.0  235.0   47.0  305.0  243.0  186.0  282.0  261.0  300.0   
17  ...    100.0  176.0   66.0  253.0  183.0  146.0  231.0  203.0  239.0   
18  ...    113.0  152.0  127.0  150.0  106.0   52.0  235.0  112.0  179.0   
19  ...      0.0   79.0  163.0  220.0  119.0  164.0  135.0  152.0  153.0   
20  ...     79.0    0.0  236.0  201.0   90.0  195.0   90.0  127.0   84.0   
21  ...    163.0  236.0    0.0  273.0  226.0  148.0  2

In [218]:
def fitness(permutation):
    totalCost=0
    for i in range(1,len(permutation)):
        totalCost+=costs[permutation[i-1]][permutation[i]]
    totalCost+=costs[permutation[-1]][permutation[0]]
    return -totalCost

def getInitialPopulation(size,count):
    return [np.random.permutation(size).tolist() for i in range(count)]

def choose(population,count):
    return heapq.nlargest(count,population,key=fitness) # better performance than sorting...

def swapMutation(chromosome):
    if random.random()>mutationPossibility : return chromosome
    a=int(random.random()*len(chromosome))
    b=int(random.random()*len(chromosome))
    (chromosome[a],chromosome[b])=(chromosome[b],chromosome[a])
    return chromosome

def orderRecombination(parent1,parent2):
    if random.random() >crossoverPossibility: return [parent1,parent2] #crossover not happened
    n=len(parent1)
    b1 =int(random.random() * n)
    b2=int(random.random() * n)
    start=min(b1,b2)
    end =max(b1,b2)
    
    child1= [gen for gen in parent2 if gen not in parent1[start:end]]
    child1[start:start]=parent1[start:end]
    
    child2=[gen for gen in parent1 if gen not in parent2[start:end]]
    child2[start:start]=parent2[start:end]
    
    child1=swapMutation(child1)
    child2=swapMutation(child2)
    
    return [child1,child2]


def rouletteGeneration(population,eliteSize):
    n=len(population)
    pool=choose(population,eliteSize)
    fits=[fitness(popula) for popula in population]
    minFitness=min(fits)
    fits=[fit-minFitness for fit in fits]
    wheel=np.cumsum(fits).tolist()
    while len(pool)<n:
        parent1=population[np.searchsorted(fits,random.random() * fits[-1])]
        parent2=population[np.searchsorted(fits,random.random() * fits[-1])]
        pool.extend(orderRecombination(parent1,parent2)) #we can use other combination methods here...
    return pool

def tournamentGeneration(population,eliteSize):
    n=len(population)
    pool=choose(population,eliteSize)
    while len(pool)<n:
        tourn1=random.sample(population,tournamentSize)
        tourn2=random.sample(population,tournamentSize)
        pool.extend(orderRecombination(max(tourn1,key=fitness),max(tourn2,key=fitness)))
    return pool 

In [224]:
initialPopulation=getInitialPopulation(numberOfCities,initialPopulationCount)
roulettePopulation=initialPopulation
tournamentPopulation=initialPopulation
rouletteBestfits=[]
tournamentBestFits=[]
counter=0
while counter<maxFrozenIteration:
#     roulettePopulation=rouletteGeneration(roulettePopulation,elitismSetSize)
    tournamentPopulation=tournamentGeneration(tournamentPopulation,elitismSetSize)
    
#     rouletteFits=[fitness(popula) for popula in roulettePopulation]
    tournamentFits=[fitness(popula) for popula in tournamentPopulation]
    
#     theRouletteBest=-max(rouletteFits)
    theTournamentBest=-max(tournamentFits)
    
#     if(len(rouletteBestfits)>0 and theRouletteBest==rouletteBestfits[-1]):counter+=1
    if(len(tournamentBestFits)>0 and theTournamentBest==tournamentBestFits[-1]):counter+=1
        
#     rouletteBestfits.append(theRouletteBest)
    tournamentBestFits.append(-max(tournamentFits))
    

plt.plot(rouletteBestfits)
plt.plot(tournamentBestFits)
plt.ylabel('fitness')
plt.xlabel('iteration')
plt.show()
print(max(tournamentPopulation,key=fitness))
print(min(tournamentPopulation,key=fitness))

[22, 6, 24, 18, 10, 21, 16, 13, 17, 14, 3, 9, 19, 1, 20, 4, 28, 2, 25, 8, 11, 5, 27, 0, 23, 12, 15, 26, 7]
[22, 6, 24, 18, 10, 11, 16, 13, 17, 14, 3, 9, 19, 1, 20, 4, 28, 2, 25, 8, 21, 5, 27, 0, 23, 12, 15, 26, 7]


In [208]:
population=getInitialPopulation(29,1000)
choose(population,100)

[[10,
  18,
  24,
  0,
  15,
  21,
  13,
  11,
  26,
  23,
  5,
  4,
  28,
  16,
  17,
  22,
  12,
  27,
  7,
  6,
  14,
  1,
  9,
  19,
  2,
  25,
  8,
  20,
  3],
 [1,
  8,
  25,
  24,
  18,
  10,
  6,
  23,
  7,
  13,
  21,
  4,
  20,
  9,
  15,
  26,
  19,
  12,
  3,
  17,
  22,
  16,
  14,
  0,
  5,
  27,
  11,
  28,
  2],
 [24,
  9,
  19,
  7,
  10,
  23,
  4,
  11,
  20,
  13,
  16,
  6,
  0,
  27,
  12,
  22,
  26,
  18,
  15,
  8,
  1,
  28,
  5,
  2,
  25,
  17,
  14,
  3,
  21],
 [23,
  20,
  0,
  22,
  6,
  21,
  3,
  2,
  19,
  17,
  28,
  12,
  25,
  1,
  5,
  14,
  26,
  18,
  24,
  27,
  7,
  15,
  13,
  16,
  10,
  9,
  11,
  4,
  8],
 [21,
  16,
  27,
  9,
  24,
  1,
  11,
  0,
  12,
  4,
  8,
  2,
  15,
  26,
  22,
  28,
  25,
  7,
  20,
  19,
  23,
  5,
  10,
  13,
  17,
  14,
  6,
  3,
  18],
 [17,
  24,
  7,
  12,
  22,
  6,
  2,
  20,
  19,
  5,
  4,
  0,
  21,
  10,
  16,
  9,
  1,
  28,
  14,
  3,
  26,
  15,
  23,
  13,
  25,
  11,
  8,
  27,
  18],
 [11,
  12